<h1><center>Mini-Project  -  Foundations of Knowledge Graphs</center></h1>
<h2><center>Trial Notebook</center></h2>
<h3><center>Knowledge Group LRJ</center></h3>
<center><b>Team members (IMT user name):</b></center>
   <center>Jonas Thorben Becker (becks100)</center>
   <center>Lukas Kneilmann (lukn)</center>
   <center>Rupesh Sapkota (rupezzz) (now deregistered)</center>
<br>
<br>
This Jupyter notebook was used to evaluate different combinations of samplers and classifiers for the learning problems in 'kg-mini-project-train_v2.ttl'. 
The knowledge gained here was then transferred to the grading task in the Jupyter notebook <font color='blue'>'grading_classification'</font>.

<br>
<br>
<br>
<br>
<br>
<br>
Necessary imports

In [7]:
from rdflib import Graph
import rdflib
from owlready2 import *
from owlrl import *
import numpy as np
import gensim
from sklearn.preprocessing import StandardScaler
from imblearn.combine import SMOTEENN
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.kernel_approximation import Nystroem
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import KMeansSMOTE
from imblearn.over_sampling import BorderlineSMOTE 
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.combine import SMOTETomek 
import pandas as pd

<br>

- Parses the graph in the file <font color='blue'>kg-mini-project-grading.ttl</font> and saves it as <font color='blue'>g</font>
- loads the Word2vec (?) Embedding and saves it as a <font color='blue'>model</font>

In [8]:
g = Graph()
g.parse('kg-mini-project-train_v2.ttl', format='n3')

# Load pre-trained OWL2Vec* model.
owl2vec_model = gensim.models.Word2Vec.load("output")

<br>
<br>
Function <font color='blue'>data</font>:  

- for the learning problem specified by parameter <font color='blue'>i</font>: 
    - returns the training data set (<font color='blue'>y_np, X_np</font>) : 
        - creates <font color='blue'>y_np</font> by determining the class of the individuals of the learning problem
        - creates <font color='blue'>x_np</font> by embedding the individuals using <font color='blue'>owl2vec_model</font> 

In [9]:
def data (i):
    lp = rdflib.Graph()
    lp_object_str = []
    lp_y = []
    
    # tag individual with class id
    s = 'https://lpbenchgen.org/resource/lp_' + str(i)
    print(s)
    for i, (s,p,o) in enumerate(g.triples((rdflib.term.URIRef(s), None, None))): 
         if str(o)=='https://lpbenchgen.org/class/LearningProblem':
            pass
         else:
            lp_object_str.append(str(o))
            if str(p) == 'https://lpbenchgen.org/property/excludesResource':
                 lp_y.append(0)
            else:
                    lp_y.append(1) 

    # embed individuals                
    lp_emb = []
    for i in range(len(lp_object_str)):
        lp_emb.append(owl2vec_model.wv[lp_object_str[i]])
    
    #to numpy
    X_np = np.array(lp_emb)
    y_np = np.array(lp_y)
    return X_np, y_np

<br>

- Evaluates the combination of different sampling methods with an exhaustive grid search over different hyperparameters in connection with cross validation for a given model for each individual learning problem in graph <font color='blue'>g</font>

- Saves the best sampling method and the best hyperparameters (based on the F1-score) for each learning problem.

- Due to the  train test split, this combination can then be tested on unseen data and the results are saved

In [10]:
scaler = StandardScaler()
l = []
modellist = []
creports =[]
smplist = []

best_clf = None

ros = RandomOverSampler(random_state=0)
smsvm = SVMSMOTE(random_state=42,n_jobs=-1)
sm = SMOTE(random_state=42)
smote_enn = SMOTEENN(random_state=0,n_jobs=-1)   # perhaps different resampling because its slow 
smt = SMOTETomek(random_state=42,n_jobs=-1)
ada = ADASYN(random_state=42)

#sampler =[ros,smsvm,sm,smote_enn,smt,ada]
sampler =[smsvm]

for i in range(1,26):
    # create X,Y
    X_np,y_np = data(i)
    

    # train test split & resample
    X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.3, random_state=42)
    
    # Standardize and resample
    scaler = StandardScaler()
    scaler = scaler.fit(X_train)
    X_train = scaler.transform(X_train) # for svm
    X_test = scaler.transform(X_test) 
    
    best_score =0
    best_smp =''
    
    for smp in sampler:
        
        try: 
            X_train_re, y_train_re = smp.fit_resample(X_train, y_train)  
            
            print(X_train.shape)

            parameters = { 'C':[0.1,0.5,1]}  
            m= LinearSVC(dual=False, penalty='l2',max_iter=2000) # insert various models

            clf = GridSearchCV(m, parameters,verbose=0,n_jobs=-1, scoring = {'precision_macro','f1', 'accuracy', 'recall','roc_auc'}, refit='f1')  
            clf.fit(X_train_re, y_train_re)
            print( str(smp) + ' sampling done, score:'+ str(clf.best_score_))
        
            if clf.best_score_ > best_score:
                best_score = clf.best_score_
                best_clf = clf
                best_smp = str(smp)
                
        except:
            print(str(smp)+ ': An exception occurred')
            
            X_train_re, y_train_re = smote_enn.fit_resample(X_train, y_train)  

            parameters = { 'C':[0.1,0.5,1]}  
            m= LinearSVC(dual=False, penalty='l2',max_iter=2000)

            clf = GridSearchCV(m, parameters,verbose=0,n_jobs=-1, scoring = {'precision_macro','f1', 'accuracy', 'recall','roc_auc'}, refit='f1')  
            clf.fit(X_train_re, y_train_re)
            print( 'smote sampling done, score:'+ str(clf.best_score_))
        
            if clf.best_score_ > best_score:
                best_score = clf.best_score_
                best_clf = clf
                best_smp = 'smote_enn'
        
    
    # save and print results
    l.append(best_clf.best_params_)
    smplist.append(best_smp)
    print(best_clf.best_params_, best_smp)
    print(classification_report(y_test, best_clf.predict(X_test))) #for final decision
    print()
    creports.append(classification_report(y_test, best_clf.predict(X_test),output_dict=True)) #for final decision
    
    best_score=0

https://lpbenchgen.org/resource/lp_1
(15660, 300)
SVMSMOTE(n_jobs=-1, random_state=42) sampling done, score:0.9985913135396565
{'C': 1} SVMSMOTE(n_jobs=-1, random_state=42)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6670
           1       0.55      0.55      0.55        42

    accuracy                           0.99      6712
   macro avg       0.77      0.77      0.77      6712
weighted avg       0.99      0.99      0.99      6712


https://lpbenchgen.org/resource/lp_2
(15660, 300)
SVMSMOTE(n_jobs=-1, random_state=42) sampling done, score:0.9995933103523722
{'C': 1} SVMSMOTE(n_jobs=-1, random_state=42)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6664
           1       0.90      0.98      0.94        48

    accuracy                           1.00      6712
   macro avg       0.95      0.99      0.97      6712
weighted avg       1.00      1.00      1.00      6712


c:\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are 

(15660, 300)
SVMSMOTE(n_jobs=-1, random_state=42) sampling done, score:1.0
{'C': 0.1} SVMSMOTE(n_jobs=-1, random_state=42)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5661
           1       1.00      1.00      1.00      1051

    accuracy                           1.00      6712
   macro avg       1.00      1.00      1.00      6712
weighted avg       1.00      1.00      1.00      6712


https://lpbenchgen.org/resource/lp_18
(15660, 300)
SVMSMOTE(n_jobs=-1, random_state=42) sampling done, score:0.9993642321240266
{'C': 0.5} SVMSMOTE(n_jobs=-1, random_state=42)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6614
           1       0.96      1.00      0.98        98

    accuracy                           1.00      6712
   macro avg       0.98      1.00      0.99      6712
weighted avg       1.00      1.00      1.00      6712


https://lpbenchgen.org/resource/lp_19
(15660,

<br>

In the following cells, the results are inspected and evaluated

In [120]:
best_clf

GridSearchCV(estimator=LinearSVC(dual=False, max_iter=2000), n_jobs=-1,
             param_grid={'C': [0.1, 0.5, 1]}, refit='f1',
             scoring={'recall', 'precision_macro', 'accuracy', 'roc_auc', 'f1'})

In [121]:
smplist

['SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_jobs=-1, random_state=42)',
 'SVMSMOTE(n_job

In [122]:
a = np.empty(0)#
for i in range(0,2):
    a = np.append(a,creports[i].get('1').get('precision'))
a.mean()

1.0

In [123]:
a = np.empty(0)#
for i in range(0,2):
    a = np.append(a,creports[i].get('1').get('f1-score'))
a.mean()

1.0

In [124]:
a = np.empty(0)#
for i in range(0,2):
    a = np.append(a,creports[i].get('macro avg').get('f1-score'))
a.mean()

1.0

In [125]:
l

[{'C': 0.1},
 {'C': 0.1},
 {'C': 0.1},
 {'C': 0.1},
 {'C': 0.1},
 {'C': 1},
 {'C': 0.1},
 {'C': 0.5},
 {'C': 0.1},
 {'C': 0.1},
 {'C': 0.1},
 {'C': 0.1},
 {'C': 0.1},
 {'C': 0.1},
 {'C': 0.1},
 {'C': 0.1},
 {'C': 0.1},
 {'C': 0.5},
 {'C': 0.1},
 {'C': 0.1},
 {'C': 0.1},
 {'C': 1},
 {'C': 0.1},
 {'C': 0.1},
 {'C': 0.1}]